In [33]:
import cv2

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

import numpy as np

from local_vars import root_folder

import os

from ipywidgets import IntProgress
from IPython.display import display

images_folder = "SkullScan1_Images"
images_fullpath = os.path.join(root_folder, images_folder)

segmentation_threshold = 0.8 
minimum_distance = 2

deep_learning_model_path = ""
deep_learning_weights_path = "C:/Users/14ams18/Documents/weights_2019-04-08_13-48-19.h5"

input_image_size = 128
input_intensity_scaling = 1 / 255.0

In [9]:
#If loading full model instead of just weights
#model = load_model(deep_learning_model_path)

#If loading weights instead of a saved model
feature_n = 512
step_size = 1e-5

def unet(pretrained_weights = None, input_size = (input_image_size, input_image_size, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(feature_n // 16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(feature_n // 16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(feature_n // 8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(feature_n // 8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(feature_n // 4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(feature_n // 4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(feature_n // 2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(feature_n // 2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(feature_n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(feature_n, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(feature_n // 2, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(feature_n // 2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(feature_n // 2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(feature_n // 4, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(feature_n // 4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(feature_n // 4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(feature_n // 8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(feature_n // 8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(feature_n // 8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(feature_n // 16, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(feature_n // 16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(feature_n // 16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = step_size), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    model.load_weights(pretrained_weights)

    return model

model = unet(deep_learning_weights_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [38]:
image_file_list = [f for f in os.listdir(images_fullpath) if f.endswith('.png')]
n = len(image_file_list)

image_data = np.zeros([n, input_image_size, input_image_size, 1]).astype(np.uint8)

print("Reading files and saving them in numpy arrays...")

for i in range(n):
    current_file_name = image_file_list[i]
    current_file_path = os.path.join(images_fullpath, current_file_name)
    image = cv2.imread(current_file_path, 0)
    if image is None:
        print("File not read: {}".format(current_file_path))
        break
    if image.shape[0] != input_image_size or image.shape[1] != input_image_size:
        image_data[i][:,:,0] = cv2.resize(image, (input_image_size, input_image_size)).astype(np.uint8)
    else:
        image_data[i][:,:,0] = image.astype(np.uint8)
    

image_data = image_data * input_intensity_scaling

Reading files and saving them in numpy arrays...


In [41]:
segmentations = model.predict(image_data)

In [44]:
for i in range(n):
    for j in range(input_image_size):
        for k in range(input_image_size):
            if segmentations[i,j,k] < segmentation_threshold:
                segmentations[i,j,k] = 0
            else:
                segmentations[i,j,k] = 1
   

In [52]:
segmentations_output_file = os.path.join(images_fullpath, "segmentations")
np.save(segmentations_output_file, segmentations)